# Installing required modules

%pip install --quiet --no-build-isolation --force-reinstall \
    "boto3" \
    "awscli" \
    "botocore" \
    "faiss-cpu" \
    "langchain" \
    "pypdf" \
    "unstructured[pdf]" \
    "sqlalchemy" \
    "pickle5" \
    "transformers"

# Connecting to aws bedrock service & get a client

In [2]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-west-2"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
os.environ["BEDROCK_ASSUME_ROLE"] = "arn:aws:iam::195364414018:role/Crossaccountbedrock"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-west-2
  Using role: arn:aws:iam::195364414018:role/Crossaccountbedrock ... successful!
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


# Creating object of embedding and llm

In [3]:
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

br_embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", 
    client=boto3_bedrock
)

br_llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=boto3_bedrock,
    model_kwargs={"temperature":0.1}
)

is_login=True
user="MV123456789"

# Load PDF files from dir and store in vectorstore

In [4]:
from langchain.document_loaders import PyPDFDirectoryLoader, PyPDFLoader, UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import FAISS
import os

chunk_size = 1000
chunk_overlap = 500
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, 
    chunk_overlap=chunk_overlap,
    length_function = len,
)


# Getting element by element text from PDF to get doc name & element page no.

In [5]:
elements = []
files = os.listdir("public/")
for file_name in files:
    if file_name.endswith(".pdf"):
        loader = UnstructuredFileLoader("public/" + file_name, mode="elements", strategy="fast")
        elements.extend(loader.load())
print("All the elements from PDF is loaded")

The PDF <_io.BufferedReader name='public/nmdmg14060.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='public/avivaplus-breakdown-cover-policy-wording.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='public/insurance_motor_car_motor_policy_booklet_241017_NMDMG10249_v3.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


All the elements from PDF is loaded


# With source and page number as unique key, joining all document text and creating pages

In [6]:
pages = [elements[0]]
for element in elements[1:]:
    if pages[-1].metadata['source'] == element.metadata['source'] and pages[-1].metadata['page_number'] == element.metadata['page_number']:
        pages[-1].page_content += " "+element.page_content
    else:
        pages.append(element)
print("All the pages are splitted")

All the pages are splitted


# spliting pages with our custome splitter & generating vector store

In [7]:
docs, metadata = [], []

for i in range(len(pages)):
    print(f'Spliting {pages[i].metadata["source"]} page number : {pages[i].metadata["page_number"]}')
    splits =  text_splitter.split_text(pages[i].page_content)
    docs.extend(splits)
    metadata.extend([{"source": pages[i].metadata["source"], "page_number": pages[i].metadata["page_number"]}] * len(splits))

pub_vs = FAISS.from_texts(
    docs,
    br_embeddings,
    metadatas=metadata,
)

print(f"pub_vs: number of elements in the index={pub_vs.index.ntotal}")

Spliting public/nmdmg14060.pdf page number : 1
Spliting public/nmdmg14060.pdf page number : 2
Spliting public/insurance-motor-car-motoring-in-europe-241017-NMDMG2918.pdf page number : 1
Spliting public/insurance-motor-car-motoring-in-europe-241017-NMDMG2918.pdf page number : 2
Spliting public/insurance-motor-car-motoring-in-europe-241017-NMDMG2918.pdf page number : 3
Spliting public/insurance-motor-car-motoring-in-europe-241017-NMDMG2918.pdf page number : 4
Spliting public/insurance-motor-car-motoring-in-europe-241017-NMDMG2918.pdf page number : 5
Spliting public/insurance-motor-car-motoring-in-europe-241017-NMDMG2918.pdf page number : 6
Spliting public/insurance-motor-car-motoring-in-europe-241017-NMDMG2918.pdf page number : 7
Spliting public/insurance-motor-car-motoring-in-europe-241017-NMDMG2918.pdf page number : 8
Spliting public/insurance-motor-car-motoring-in-europe-241017-NMDMG2918.pdf page number : 9
Spliting public/insurance-motor-car-motoring-in-europe-241017-NMDMG2918.pdf pa

# Createing vector store for private document 

In [8]:
loader = UnstructuredFileLoader("private/policy_certifcate_multiple_vehicle.pdf", mode="elements", strategy="fast")
elements = loader.load()
print("All the elements from PDF is loaded")

pages = [elements[0]]
for element in elements[1:]:
    if pages[-1].metadata['source'] == element.metadata['source'] and pages[-1].metadata['page_number'] == element.metadata['page_number']:
        pages[-1].page_content += " "+element.page_content
    else:
        pages.append(element)
print("All the pages are splitted")

        
docs, metadata = [], []

for i in range(len(pages)):
    print(f'Spliting {pages[i].metadata["source"]} page number : {pages[i].metadata["page_number"]}')
    splits =  text_splitter.split_text(pages[i].page_content)
    docs.extend(splits)
    metadata.extend([{"source": pages[i].metadata["source"], "page_number": pages[i].metadata["page_number"]}] * len(splits))

pvt_vs = FAISS.from_texts(
    docs,
    br_embeddings,
    metadatas=metadata,
)

print(f"pvt_vs: number of elements in the index={pvt_vs.index.ntotal}")

All the elements from PDF is loaded
All the pages are splitted
Spliting private/policy_certifcate_multiple_vehicle.pdf page number : 1
Spliting private/policy_certifcate_multiple_vehicle.pdf page number : 2
Spliting private/policy_certifcate_multiple_vehicle.pdf page number : 3
Spliting private/policy_certifcate_multiple_vehicle.pdf page number : 4
Spliting private/policy_certifcate_multiple_vehicle.pdf page number : 5
Spliting private/policy_certifcate_multiple_vehicle.pdf page number : 6
Spliting private/policy_certifcate_multiple_vehicle.pdf page number : 7
Spliting private/policy_certifcate_multiple_vehicle.pdf page number : 8
Spliting private/policy_certifcate_multiple_vehicle.pdf page number : 9
Spliting private/policy_certifcate_multiple_vehicle.pdf page number : 10
Spliting private/policy_certifcate_multiple_vehicle.pdf page number : 11
Spliting private/policy_certifcate_multiple_vehicle.pdf page number : 12
Spliting private/policy_certifcate_multiple_vehicle.pdf page number : 

# Save vector store for later use 

In [9]:
# import pickle
# print(type(pub_vs))
# with open("pub_vs.pkl", "wb") as f:
#     pickle.dump(pub_vs, f)

# Adding index wapper to vector store for faster querying

In [35]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
pub_vs_index = VectorStoreIndexWrapper(vectorstore=pub_vs)
# print("Test run query in pub_vs_index:")
# query = "What are the vehicles covered in the policy?"
# print_ww(pub_vs_index.query(f"'{query}' is this question answerable ? please return only 'TRUE' keyword if it is answerable else 'FALSE' keyword", llm=br_llm))

pvt_vs_index = VectorStoreIndexWrapper(vectorstore=pvt_vs)
# print("Test run query in pvt_vs_index:")
# print_ww(pvt_vs_index.query(f"'{query}' is this question answerable ? please return only 'TRUE' keywrod if it is answerable else 'FALSE' keyword", llm=br_llm))

Test run query in pub_vs_index:
 Based on the provided context, this question is answerable. The policy covers the following
vehicles:

- The motor vehicle described in your schedule.

- Any motor vehicle loaned to a permitted driver as shown on your certificate of motor insurance by
a supplier nominated by the insurer following a claim.

- Any motor vehicle not exceeding 3.5 tonnes gross vehicle weight loaned to a permitted driver for
up to 7 days by a garage, motor engineer or vehicle repairer while the insured motor vehicle is
being serviced, repaired or having an MOT test.

Therefore, the answer is TRUE.
Test run query in pvt_vs_index:
 TRUE


# Creating a function to extract context from pub and pvt PDFS.

In [58]:
def get_pub_context(query):
    print("Getting PDF context from pub_vs_index")
    pub_context=""
    for k, i in enumerate(pub_vs_index.vectorstore.similarity_search(query, k=1)):
        # pub_context += f"Source Document - {i.metadata['source']}\nPage Number - {i.metadata['page_number']}\nContext - {i.page_content}\n"
        pub_context = {
            "source": i.metadata['source'],
            "page_number": i.metadata['page_number'],
            "context": i.page_content,
        }
    return pub_context

def get_pvt_context(query):
    print("Getting PDF context from pvt_vs_index")
    pvt_context = ""
    for k, i in enumerate(pvt_vs_index.vectorstore.similarity_search(query, k=1)):
        # pvt_context += f"Source Document - {i.metadata['source']}\nPage Number - {i.metadata['page_number']}\nContext - {i.page_content}\n"
        pvt_context = {
            "source": i.metadata['source'],
            "page_number": i.metadata['page_number'],
            "context": i.page_content,
        }
    return pvt_context

def get_pdf_context(query):
    
    search_text = ["I", "MY", "MYSELF", "WE", "MINE", "ME", "OUR", "OURS", "HIMSELF", "HERSELF"]
    for search in search_text:
        if search in query.upper().split(" "):
            pvt_context = get_pvt_context(query)
            return [pvt_context]
        
    template = """
    You are TRUE or FALSE answering bot, you will only retrun keyword 'TRUE' if the user query is answerable else 'FALSE'.
    User Query : {query}
    """
    pub_result = pub_vs_index.query(template.format(query=query), llm=br_llm)
    # print(f"pub_result : {pub_result}")
    pvt_result = pvt_vs_index.query(template.format(query=query), llm=br_llm)
    # print(f"pvt_result : {pvt_result}")
    if "TRUE" in str(pvt_result) and "TRUE" in str(pub_result):
        pub_context = get_pub_context(query)
        pvt_context = get_pvt_context(query)
        return [pvt_context, pub_context]
    elif "TRUE" in str(pvt_result) and "FALSE" in str(pub_result):
        pvt_context = get_pvt_context(query)
        return [pvt_context]
    elif "FALSE" in str(pvt_result) and "TRUE" in str(pub_result):
        pub_context = get_pub_context(query)
        return [pub_context]
    else:
        return  [{
            "source": "NA",
            "page_number": 0,
            "context": "This is out of context!!!",
        }]

In [60]:
query = " I've had an accident, car will be off the road for 3 weeks to repair. 'Does my insurance have courtesy car cover'?"
context = get_pdf_context(query)
print(len(context))
print(context)

Getting PDF context from pvt_vs_index
1
[{'source': 'private/policy_certifcate_multiple_vehicle.pdf', 'page_number': 11, 'context': 'vehicles’ section will show which cover you have. Comprehensive Loss of or damage to your vehicle - Non manufacturer fitted accessories limit - Manufacturer fitted accessories limit - New vehicle replacement Your liability - Third party property damage limit - Third party property damage limit - hazardous goods and locations - Terrorism limit - Claimant costs limit Motor Injury Protection - Single claim limit - Annual claim limit Medical Expenses - Medical Expenses limit Personal Belongings - Personal Belongings limit Child Seat - Child Seat limit Tool Cover - Tool cover limit Vehicle Recovery - Vehicle Recovery due to illness Replacement Locks - Replacement Locks Glass - Approved Repairer windscreen replacement limit - Non approved Repairer windscreen replacement limit - Windscreen replacement excess - Windscreen repair excess Uninsured Driver Promise - 

# Creating Chat History Memory

In [61]:
from langchain.memory import ConversationBufferWindowMemory, ConversationSummaryMemory # llm 

# store previous interactions using ConversationalBufferMemory and add custom prompts to the chat.
memory = ConversationBufferWindowMemory(
    # llm=br_llm,
    input_key="customer_query",
    memory_key="history", 
    return_messages=False, 
    k=3,
    ai_prefix="Assistant",
    human_prefix="Human",
)

# Prompt template for chain

In [62]:
from langchain.prompts.prompt import PromptTemplate

_one_context_template = """
{history}

Human: In the context of "{context}", can you answer on the below question "{customer_query}" and copy them out word-for-word from the context. If you couldn't find any relevant context, please reply to customer politely that you don't know the answer to the question could they mind rephrasing the question.

Assistant:
"""
ONE_CONTEXT_PROMPT = PromptTemplate(template=_one_context_template, input_variables=["customer_query", "context", "history"])

In [63]:
from langchain.prompts.prompt import PromptTemplate

_two_context_template = """
{history}

Human: In the context of "{pvt_context}" and "{pub_context}", can you answer on the below question "{customer_query}" and copy them out word-for-word from the context. First provide insights from "{pvt_context}" then discuss infomation from "{pub_context}".

Assistant:
"""
TWO_CONTEXT_PROMPT = PromptTemplate(template=_two_context_template, input_variables=["customer_query", "pub_context", "pvt_context", "history"])

In [64]:
# turn verbose to True to see the full logs and documents
from langchain.chains import LLMChain
    
def ask_copilot(query):
    context = get_pdf_context(query)
    if len(context) == 1:
        qa = LLMChain(
            llm=br_llm, 
            verbose=False, 
            prompt=ONE_CONTEXT_PROMPT,
            memory=memory,
        )
        ai_result = qa.predict(customer_query=query, context=context[0]['context'])
        return_result = {
            "result_message":str(ai_result),
            "source":[context[0]['source']],
            "page_number":[context[0]['page_number']],
            "search_text":[context[0]['context']],
        }
    elif len(context) == 2:
        qa = LLMChain(
            llm=br_llm, 
            verbose=False, 
            prompt=TWO_CONTEXT_PROMPT,
            memory=memory,
        )
        ai_result = qa.predict(customer_query=query, pvt_context=context[0]['context'], pub_context=context[1]['context'])
        return_result = {
            "result_message":str(ai_result),
            "source":[context[0]['source'], context[1]['source']],
            "page_number":[context[0]['page_number'], context[1]['page_number']],
            "search_text":[context[0]['context'], context[1]['context']],
        }
    print(f"AI Result = {return_result['result_message']}\n")
    print(f"Source Document = {return_result['source']}\n")
    print(f"Page Number = {return_result['page_number']}\n")
    print(f"Search Text = {return_result['search_text']}\n")
    return return_result

In [65]:
result = ask_copilot("What is the name of Principal policyholder?")

Getting PDF context from pvt_vs_index
AI Result =  The name of the Principal policyholder is Mr. Test Tester.

Source Document = ['private/policy_certifcate_multiple_vehicle.pdf']

Page Number = [10]

Search Text = ['Principal policyholder: Mr. Test Tester Address: 77 Music House Lane, Norwich, NR1 1QN Contact number: 07459617348 Policy number: MMV070055371 Period of insurance: 14.09 on 27 September 2023 to 23.59 on 26 September 2024 Your total premium is £3,677.00 Payment Method - Annual This includes Insurance Premium Tax (IPT) at the appropriate rate where applicable. This premium is based on the information you have provided. Any change affecting any vehicles could result in a change of premium. Your Demands and Needs Statement Following your enquiry, the information detailed below indicates the demands and needs you stated for your insurance requirements. Important Information You should note that we have not taken into account any existing insurance arrangements you may have. Ple

In [66]:
result = ask_copilot("What will be my Cancellation fees and charges?")

Getting PDF context from pvt_vs_index
AI Result =  Cancellation fee £38 (exc IPT)

Source Document = ['private/policy_certifcate_multiple_vehicle.pdf']

Page Number = [12]

Search Text = ["-*- Demonstration Powered by OpenText Exstream 09/27/2023, Version 16.6.32 32-bit -*- Your policy schedule - continued Young driver excess - Young driver excess - under 21's - Young driver excess - 21-24 year olds Non approved repairer excess - Non approved repairer excess £300 per claim £200 per claim £200 per claim Your fees Cancellation fee £38 (exc IPT) Clauses applicable The following are special terms and conditions which should be read in conjunction with your Policy Booklet. Clauses applicable to the Vehicles section Opt out of Courtesy Vehicle Benefit Courtesy Vehicle cover for vehicle registration numbers(s) AU53ZWM and A1 has been removed from Section 1 of the policy. Aviva Insurance Limited. Registered in Scotland, No. 2116. Registered Office: Pitheavlis, Perth PH2 0NH. Authorised by the 

In [67]:
result = ask_copilot(" I've had an accident, car will be off the road for 3 weeks to repair. 'Does my insurance have courtesy car cover'?")

Getting PDF context from pvt_vs_index
AI Result =  Unfortunately I do not have enough context to definitively answer whether your insurance policy includes courtesy car cover. Could you please rephrase the question or provide more details? I want to make sure I give you an accurate answer.

Source Document = ['private/policy_certifcate_multiple_vehicle.pdf']

Page Number = [11]

Search Text = ['vehicles’ section will show which cover you have. Comprehensive Loss of or damage to your vehicle - Non manufacturer fitted accessories limit - Manufacturer fitted accessories limit - New vehicle replacement Your liability - Third party property damage limit - Third party property damage limit - hazardous goods and locations - Terrorism limit - Claimant costs limit Motor Injury Protection - Single claim limit - Annual claim limit Medical Expenses - Medical Expenses limit Personal Belongings - Personal Belongings limit Child Seat - Child Seat limit Tool Cover - Tool cover limit Vehicle Recovery -

In [69]:
result = ask_copilot(" A named driver is added on the policy - 'Can they drive the vehicle'?")

Getting PDF context from pub_vs_index
AI Result =  Based on the context provided, the answer is:

"Yes, a named driver on the policy can drive the damaged vehicle, as long as the certificate of motor insurance allows that person to drive the motor vehicle."

Source Document = ['public/insurance_motor_car_motor_policy_booklet_241017_NMDMG10249_v3.pdf']

Page Number = [6]

Search Text = ['that person to drive the motor vehicle. Where the vehicle policyholder is the owner/registered keeper of the vehicle any benefits which may be due under Sections 1, 3, 6, 7, 9 and 11 may be paid, at our discretion to them instead of the principal policyholder named in your schedule. Named driver Person nominated by the principal policyholder as being a user (but not the main user) of any motor vehicle insured under this policy as described in your schedule, providing the certificate of motor insurance allows that person to drive the motor vehicle (Named drivers are not entitled to benefits provided unde